El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Importacion de librerias

In [2]:
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor


## Preparación de datos

In [3]:
df = pd.read_csv('/datasets/car_data.csv')

df.info()
print()
display(df.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
131723,15/03/2016 10:39,4400,small,2000,manual,169,clio,150000,0,petrol,renault,NaN,15/03/2016 00:00,0,75031,05/04/2016 19:17
108007,31/03/2016 18:43,2999,sedan,2002,manual,170,5er,150000,3,NaN,bmw,NaN,31/03/2016 00:00,0,63450,04/04/2016 11:17
298836,02/04/2016 19:39,4500,small,2011,manual,60,fox,90000,12,petrol,volkswagen,no,02/04/2016 00:00,0,22459,05/04/2016 19:44
226216,08/03/2016 00:55,0,sedan,1994,manual,102,3er,150000,7,NaN,bmw,no,07/03/2016 00:00,0,97980,11/03/2016 17:18
130599,21/03/2016 18:49,6400,small,2011,manual,70,ibiza,70000,2,petrol,seat,no,21/03/2016 00:00,0,67361,30/03/2016 07:16
87423,23/03/2016 00:56,3000,sedan,2001,auto,184,NaN,150000,10,gasoline,bmw,NaN,22/03/2016 00:00,0,32423,24/03/2016 14:16
238532,19/03/2016 15:39,450,NaN,1997,NaN,0,vectra,150000,0,NaN,opel,no,19/03/2016 00:00,0,32130,06/04/2016 23:45
285480,31/03/2016 19:38,300,small,1996,manual,70,polo,150000,0,petrol,volkswagen,NaN,31/03/2016 00:00,0,88682,06/04/2016 13:45
349162,19/03/2016 21:36,7000,sedan,2009,manual,0,b_klasse,150000,11,gasoline,mercedes_benz,NaN,19/03/2016 00:00,0,23554,21/03/2016 02:46
73482,11/03/2016 13:37,2900,sedan,2005,NaN,0,golf,150000,10,gasoline,volkswagen,NaN,11/03/2016 00:00,0,4177,11/03/2016 13:37


In [4]:
df.describe().round(2)
    

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.00,354369.00,354369.00,354369.00,354369.00,354369.0,354369.00
mean,4416.66,2004.23,110.09,128211.17,5.71,0.0,50508.69
std,4514.16,90.23,189.85,37905.34,3.73,0.0,25783.10
min,0.00,1000.00,0.00,5000.00,0.00,0.0,1067.00
25%,1050.00,1999.00,69.00,125000.00,3.00,0.0,30165.00
50%,2700.00,2003.00,105.00,150000.00,6.00,0.0,49413.00
75%,6400.00,2008.00,143.00,150000.00,9.00,0.0,71083.00
max,20000.00,9999.00,20000.00,150000.00,12.00,0.0,99998.00


In [5]:
df.columns = df.columns.str.lower()
print(df.columns)

Index(['datecrawled', 'price', 'vehicletype', 'registrationyear', 'gearbox',
       'power', 'model', 'mileage', 'registrationmonth', 'fueltype', 'brand',
       'notrepaired', 'datecreated', 'numberofpictures', 'postalcode',
       'lastseen'],
      dtype='object')


In [6]:
# Cambio a valor fecha

def cambio_fecha(df):
    columnas_fechas = ['datecrawled', 'datecreated', 'lastseen']
    
    for col in columnas_fechas:
        df[col] = pd.to_datetime(df[col])
    
    return df

df = cambio_fecha(df)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   datecrawled        354369 non-null  datetime64[ns]
 1   price              354369 non-null  int64         
 2   vehicletype        316879 non-null  object        
 3   registrationyear   354369 non-null  int64         
 4   gearbox            334536 non-null  object        
 5   power              354369 non-null  int64         
 6   model              334664 non-null  object        
 7   mileage            354369 non-null  int64         
 8   registrationmonth  354369 non-null  int64         
 9   fueltype           321474 non-null  object        
 10  brand              354369 non-null  object        
 11  notrepaired        283215 non-null  object        
 12  datecreated        354369 non-null  datetime64[ns]
 13  numberofpictures   354369 non-null  int64   

In [7]:
porcentaje_nulos = ((df.isnull()).sum() / len(df)) * 100

print(f'Porcentaje de valores nulos por columna:\n --------------------------------------- \n{porcentaje_nulos.round(2)}')


Porcentaje de valores nulos por columna:
 --------------------------------------- 
datecrawled           0.00
price                 0.00
vehicletype          10.58
registrationyear      0.00
gearbox               5.60
power                 0.00
model                 5.56
mileage               0.00
registrationmonth     0.00
fueltype              9.28
brand                 0.00
notrepaired          20.08
datecreated           0.00
numberofpictures      0.00
postalcode            0.00
lastseen              0.00
dtype: float64


In [8]:
duplicados = df.duplicated().sum()
print(f'Valores duplicados en la base de datos: \n{duplicados}')

Valores duplicados en la base de datos: 
262


In [9]:
# Limpieza del DataFrame

# Borramos los duplicados
df = df.drop_duplicates()

# Outliers
df = df[(df['price'] >100) & (df['price']<100000)]

categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numeric_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != 'price']

for col in categorical_cols:
    df[col] = df[col].fillna('Unknown')

for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())


In [10]:
print(f"Total de nulos después de limpieza: {df.isnull().sum().sum()}")
print(f"¿Hay duplicados? {df.duplicated().sum()}")

Total de nulos después de limpieza: 0
¿Hay duplicados? 0


<div class="alert alert-block alert-success">  
<b>Comentario del revisor</b> <a class="tocSkip"></a><br>  
<b>Éxito</b> – Tu proceso de carga, inspección y limpieza inicial muestra un dominio claro del flujo de preparación de datos. La conversión de fechas, el manejo de nulos y la depuración del dataset están planteados de forma ordenada y consistente. Buen ritmo en esta etapa del análisis.  
</div>


## Entrenamiento del modelo 

In [11]:
target = df['price']
features = df.drop(['price'], axis=1)

X_train_df, x_test_df, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state=12345)

print(f'Dimension de entrenamiento: {X_train_df.shape}')
print(f'Dimension de prueba: {x_test_df.shape}')

Dimension de entrenamiento: (271812, 15)
Dimension de prueba: (67953, 15)


In [12]:
# Repetimos el proceso para X_train
cat_cols = X_train_df.select_dtypes(include=['object']).columns.tolist()
num_cols = X_train_df.select_dtypes(include=['int64','float64']).columns.tolist()
num_cols = [c for c in num_cols if c != ['price']]

# Escalamos las columnas numericas
scaler = StandardScaler()

X_train_num = scaler.fit_transform(X_train_df[num_cols])
X_test_num = scaler.transform(x_test_df[num_cols])


# Aplicamos OHE para las columnas categoricas
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse=False)
X_train_cat = ohe.fit_transform(X_train_df[cat_cols])
X_test_cat = ohe.transform(x_test_df[cat_cols])

X_train = np.hstack([X_train_num, X_train_cat])
X_test = np.hstack([X_test_num, X_test_cat])

print(f'Set de entrenamiento final: {X_train.shape}')
print(f'Set de prueba final: {X_test.shape}')

Set de entrenamiento final: (271812, 320)
Set de prueba final: (67953, 320)


<div class="alert alert-block alert-success">  
<b>Comentario del revisor</b> <a class="tocSkip"></a><br>  
<b>Éxito</b> – El proceso de división, transformación y construcción de matrices finales está bien estructurado y muestra control sobre la preparación del conjunto para el modelado. Vas avanzando con firmeza en esta etapa técnica.  
</div>


## Análisis del modelo

In [13]:
### Evaluacion del modelo

def evaluar_modelo(nombre, modelo, X_train, y_train, X_test, y_test):
    inicio = time.time()
    modelo.fit(X_train, y_train)
    fin = time.time()

    pred_train = modelo.predict(X_train)
    pred_test = modelo.predict(X_test)

    rmse_train = mean_squared_error(y_train, pred_train, squared=False)
    rmse_test = mean_squared_error(y_test, pred_test, squared=False)

    print(f'=== {nombre} ===')
    print(f'Tiempo de entrenamiento: {fin - inicio:.2f} s')
    print(f'RMSE train: {rmse_train:.2f}')
    print(f'RMSE test: {rmse_test:.2f}')
    print()

    return {
        'modelo': nombre,
        'tiempo': fin - inicio,
        'rmse_train': rmse_train,
        'rmse_test': rmse_test
    }

### Regreion Lineal (base)

In [14]:
resultados = []

linear_regression = LinearRegression()
resultado_lin = evaluar_modelo('Linear Regression', linear_regression, X_train, y_train, X_test, y_test)
resultados.append(resultado_lin)

=== Linear Regression ===
Tiempo de entrenamiento: 9.55 s
RMSE train: 3106.93
RMSE test: 3108.17



### Arbol de Decision

In [15]:
mejor_rmse_tree = float('inf')
mejor_params_tree = None

for max_depth in [5,10,20, None]:
    tree = DecisionTreeRegressor(
        random_state = 12345, 
        max_depth = max_depth
    )
    nombre = f'Arbol de Decision (max_depth ={max_depth})'
    
    resultado_tree = evaluar_modelo(
        nombre, 
        tree, 
        X_train,
        y_train,
        X_test,
        y_test
    )
    resultados.append(resultado_tree)

    if resultado_tree['rmse_test'] < mejor_rmse_tree:
        mejor_rmse_tree = resultado_tree['rmse_test']
        mejor_params_tree = {'max_depth': max_depth}

print(f'Mejor Arbol: {mejor_params_tree}, RECM test: {mejor_rmse_tree}')
print()

=== Arbol de Decision (max_depth =5) ===
Tiempo de entrenamiento: 4.16 s
RMSE train: 2485.78
RMSE test: 2491.73

=== Arbol de Decision (max_depth =10) ===
Tiempo de entrenamiento: 5.97 s
RMSE train: 1958.05
RMSE test: 2045.84

=== Arbol de Decision (max_depth =20) ===
Tiempo de entrenamiento: 7.50 s
RMSE train: 1092.87
RMSE test: 1957.17

=== Arbol de Decision (max_depth =None) ===
Tiempo de entrenamiento: 8.55 s
RMSE train: 79.36
RMSE test: 2080.04

Mejor Arbol: {'max_depth': 20}, RECM test: 1957.1669310734878



### Bosque Aleatorio

In [16]:
mejor_rmse_rf = float('inf')
mejor_params_rf = None

for est in [50,100]:
    for max_depth in [10,20]:
        rf = RandomForestRegressor(
            random_state = 12345, 
            max_depth = max_depth, 
            n_estimators = est,
            n_jobs = -1
        )
        nombre = f'Bosque Aleatorio (n_estimators = {est} | depth = {max_depth})'
        
        resultado_rf = evaluar_modelo(
            nombre, 
            rf,
            X_train,
            y_train,
            X_test,
            y_test
        )
        resultados.append(resultado_rf)
    
        if resultado_rf['rmse_test'] < mejor_rmse_rf:
            mejor_rmse_rf = resultado_rf['rmse_test']
            mejor_params_rf = {'n_estimators':est, 'max_depth': max_depth}

print(f'Mejor Bosque Aleatorio: {mejor_params_rf}, RECM test: {mejor_rmse_rf}')
print()

=== Bosque Aleatorio (n_estimators = 50 | depth = 10) ===
Tiempo de entrenamiento: 86.95 s
RMSE train: 1872.01
RMSE test: 1955.74

=== Bosque Aleatorio (n_estimators = 50 | depth = 20) ===
Tiempo de entrenamiento: 108.98 s
RMSE train: 1047.25
RMSE test: 1629.97

=== Bosque Aleatorio (n_estimators = 100 | depth = 10) ===
Tiempo de entrenamiento: 175.19 s
RMSE train: 1871.22
RMSE test: 1955.03

=== Bosque Aleatorio (n_estimators = 100 | depth = 20) ===
Tiempo de entrenamiento: 219.69 s
RMSE train: 1041.61
RMSE test: 1625.77

Mejor Bosque Aleatorio: {'n_estimators': 100, 'max_depth': 20}, RECM test: 1625.7722977227845



### LightGBM

In [17]:

mejor_rmse_lgbm = float('inf')
mejor_params_lgbm = None

for num_leaves in [30, 60]:
    for learning_rate in [0.05, 0.1]:
        lgbm = LGBMRegressor(
            random_state = 12345, 
            n_estimators = 150, 
            num_leaves = num_leaves, 
            learning_rate = learning_rate,
            verbose = -1,
            n_jobs = -1
        )
        nombre = f'LightGBM (num_leaves = {num_leaves} | learning_rate = {learning_rate})'
        
        resultado_lgbm = evaluar_modelo(
            nombre, 
            lgbm, 
            X_train,
            y_train,
            X_test,
            y_test
        )
        resultados.append(resultado_lgbm)
    
        if resultado_lgbm['rmse_test'] < mejor_rmse_lgbm:
            mejor_rmse_lgbm = resultado_lgbm['rmse_test']
            mejor_params_lgbm = {'num_leaves':num_leaves, 'learning_rate': learning_rate}

print(f'Mejor LightGBM: {mejor_params_lgbm}, RECM test: {mejor_rmse_lgbm:.2f}')
print()


=== LightGBM (num_leaves = 30 | learning_rate = 0.05) ===
Tiempo de entrenamiento: 4.00 s
RMSE train: 1749.93
RMSE test: 1772.25

=== LightGBM (num_leaves = 30 | learning_rate = 0.1) ===
Tiempo de entrenamiento: 3.71 s
RMSE train: 1669.72
RMSE test: 1706.37

=== LightGBM (num_leaves = 60 | learning_rate = 0.05) ===
Tiempo de entrenamiento: 5.29 s
RMSE train: 1664.61
RMSE test: 1705.09

=== LightGBM (num_leaves = 60 | learning_rate = 0.1) ===
Tiempo de entrenamiento: 4.70 s
RMSE train: 1585.07
RMSE test: 1654.84

Mejor LightGBM: {'num_leaves': 60, 'learning_rate': 0.1}, RECM test: 1654.84



<div class="alert alert-block alert-success">  
<b>Comentario del revisor</b> <a class="tocSkip"></a><br>  
<b>Éxito</b> – La evaluación comparativa de modelos está bien organizada y refleja un manejo seguro de métricas, tiempos y parámetros. Buen desempeño al analizar el comportamiento de cada algoritmo.  
</div>


## Resumen de resultados

In [18]:
res_df = pd.DataFrame(resultados)

# Ver qué columnas tiene
print(res_df.columns)

# Ordenar por rmse_test (de menor a mayor) y reiniciar índice
res_df_sorted = res_df.sort_values(by='rmse_test', ascending=True).reset_index(drop=True)
# Ordenar por tiempo de procesamiento (de menor a mayor) y reiniciar índice
res_df_sorted_time = res_df.sort_values(by='tiempo', ascending=True).reset_index(drop=True)

display(res_df_sorted.head(5).round(2))
print()
display(res_df_sorted_time.head(5).round(2))

Index(['modelo', 'tiempo', 'rmse_train', 'rmse_test'], dtype='object')


,modelo,tiempo,rmse_train,rmse_test
0,Bosque Aleatorio (n_estimators = 100 | depth =...,219.69,1041.61,1625.77
1,Bosque Aleatorio (n_estimators = 50 | depth = 20),108.98,1047.25,1629.97
2,LightGBM (num_leaves = 60 | learning_rate = 0.1),4.70,1585.07,1654.84
3,LightGBM (num_leaves = 60 | learning_rate = 0.05),5.29,1664.61,1705.09
4,LightGBM (num_leaves = 30 | learning_rate = 0.1),3.71,1669.72,1706.37


,modelo,tiempo,rmse_train,rmse_test
0,LightGBM (num_leaves = 30 | learning_rate = 0.1),3.71,1669.72,1706.37
1,LightGBM (num_leaves = 30 | learning_rate = 0.05),4.00,1749.93,1772.25
2,Arbol de Decision (max_depth =5),4.16,2485.78,2491.73
3,LightGBM (num_leaves = 60 | learning_rate = 0.1),4.70,1585.07,1654.84
4,LightGBM (num_leaves = 60 | learning_rate = 0.05),5.29,1664.61,1705.09


# Observaciones

Nuestro modelo base de Regresion Lineal, nos plantea como base un tiempo de carga de 9.55 segundos, con un error de entrenamiento y prueba de 3106.9 y 3108.17 respectivamente. Tomando en cuenta estos resultados a superar, se realizo una funcion de evaluacion de modelo y entrenamiento, en el que recibe los parametros necesarios para determinar el nombre, el modelo, las caracteristicas y objetivo, para finalmente devolver el nombre del modelo (mostrando ciertas caracteristicas, dependiendo el modelo), el tiempo en el que realizo el procesamiento y los resultados de la raiz del error cuadratica media (rmse) de entrenamiento y prueba.

Previo al entrenamiento de los modelos, se hizo una inspeccion de los datos y se observo que existian datos duplicados y datos nulos, al igual que se filtro la data para que no tuviera valores atipicos. Los duplicados se abordo de manera que se eliminaran completamente para que los modelos no tuvieran un desajuste  y los nulos, en sus columnas numericas, se relleno con la media de los datos y para las categoricas, con el valor 'unknown' o sea 'desaconocido'. Posteriormente, los valores numericos se escalaron para que tuvieran un peso que no desajustara los valores e importancia de los datos y para las categoricos, se realizo el meto de One Hot Encoding para codificar los valores y que se repreasentaran en numeros.

# Conclusiones 

- Linear Regression 
- Tiempo de entrenamiento: 9.55 s
- RMSE train: 3106.93
- RMSE test: 3108.17

Todos los modelos basados en árboles superan por un amplio margen a la regresión lineal.

La mejora es notoria, reduciendo casi a la mitad el error:

- 3108 → 1625 RMSE

- Esto indica una relación no lineal fuerte entre las características y el precio.

El mejor modelo global es:

Random Forest (n_estimators=100, max_depth=20)
con RMSE test = 1625.77.

Tiempo de procesamiento

El Random Forest top obtuvo:

- RMSE train: 1041.61
- RMSE test: 1625.77

Hay cierta brecha moderada, pero el rendimiento en test sigue siendo muy superior a todos los demás modelos, por lo que generaliza razonablemente bien.

LightGBM muestra incluso menos tiempo de procesamiento, pero un RMSE ligeramente mayor.

<div class="alert alert-block alert-success">  
<b>Comentario del revisor</b> <a class="tocSkip"></a><br>  
<b>Éxito</b> – Presentas un cierre analítico claro, comparando desempeño, tiempo y generalización de cada modelo. La interpretación de resultados está bien enfocada y demuestra una lectura sólida del comportamiento de los algoritmos. Buena síntesis en esta etapa final del análisis.  
</div>


# Comentario General del Revisor

<div class="alert alert-block alert-success">  
<b>Comentario del revisor</b> <a class="tocSkip"></a>  
    
Has desarrollado un proyecto con una estructura sólida y una ejecución coherente en todas las etapas del proceso analítico. Para este cierre, la metodología más adecuada es **por etapas del proceso analítico**, ya que tu trabajo integra limpieza, transformación, modelado y evaluación comparativa de forma completa.

### Preparación de datos

* Mostraste un enfoque ordenado al inspeccionar, depurar y transformar la información.
* La conversión de fechas, el manejo de duplicados y la imputación de nulos estuvieron bien aplicados, asegurando un dataset consistente.

### Análisis y transformación

* El proceso de escalado y codificación estuvo bien resuelto, habilitando un flujo adecuado para modelos lineales y basados en árboles.
* Las particiones y estructuras finales del conjunto de entrenamiento evidencian control de la preparación previa al modelado.

### Modelado y evaluación

* Comparaste varias familias de algoritmos con criterios claros: tiempo, overfitting y RMSE.
* La selección del modelo final se justificó con evidencia cuantitativa, mostrando criterio técnico en la interpretación de resultados.

### Conclusiones

* Elaboraste un cierre interpretativo que resume adecuadamente el rendimiento de cada modelo y la lógica detrás de la elección óptima.
* La lectura de los patrones no lineales en la relación entre las variables y el precio estuvo bien fundamentada.

Tu trabajo demuestra un manejo consistente del flujo completo de un proyecto de predicción con modelos supervisados. Continúa avanzando con esta claridad técnica y enfoque analítico.

</div>


# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter